In [55]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import plotly.express as px
import numpy as np

from scripts.accent_cleaner import AccentCleaner
from scripts.column_aligner import ColumnAligner
from scripts.utils import split_at_char, replace_with

In [56]:
df_raw = pd.read_csv("../data/raw/68542.csv", sep=";", decimal=",")

In [57]:
df_raw.head()

,National Total,Provinces,Municipalities,Sex,Age,Periodo,Total
0,National Total,NaN,NaN,Males,All ages,2024,23.826.871
1,National Total,NaN,NaN,Males,All ages,2023,23.565.593
2,National Total,NaN,NaN,Males,All ages,2022,23.288.747
3,National Total,NaN,NaN,Males,All ages,2021,23.248.611
4,National Total,NaN,NaN,Males,0 years old,2024,164.763


In [58]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10018440 entries, 0 to 10018439
Data columns (total 7 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   National Total  object
 1   Provinces       object
 2   Municipalities  object
 3   Sex             object
 4   Age             object
 5   Periodo         int64 
 6   Total           object
dtypes: int64(1), object(6)
memory usage: 535.0+ MB


In [59]:
df_raw.sample(15)

,National Total,Provinces,Municipalities,Sex,Age,Periodo,Total
3335929,National Total,18 Granada,18034 Cacín,Females,29 years old,2023,1
871317,National Total,06 Badajoz,06075 Magacela,Total,58 years old,2023,7
4627951,National Total,24 León,24164 Sena de Luna,Males,0 years old,2021,1
1765409,National Total,09 Burgos,09345 Santa Cruz de la Salceda,Males,99 years old,2023,0
3669328,National Total,19 Guadalajara,19155 Irueste,Total,45 years old,2024,1
6929926,National Total,37 Salamanca,37297 Santibáñez de Béjar,Total,10 years old,2022,7
5155356,National Total,"26 Rioja, La",26142 San Vicente de la Sonsierra,Total,68 years old,2024,9
48430,National Total,01 Araba/Álava,01052 Samaniego,Females,70 years old,2022,2
9150149,National Total,47 Valladolid,47182 Valdestillas,Females,84 years old,2023,5
6275334,National Total,34 Palencia,34045 Capillas,Total,72 years old,2022,0


In [60]:
df_raw.drop(columns=['National Total'], inplace=True)

In [61]:
df_raw.isnull().sum()

Provinces          1224
Municipalities    64872
Sex                   0
Age                   0
Periodo               0
Total               918
dtype: int64

In [62]:
df_municipalities_null = df_raw[df_raw['Municipalities'].isnull()]
df_municipalities_null[df_municipalities_null["Provinces"] == "08 Barcelona"]

,Provinces,Municipalities,Sex,Age,Periodo,Total
1064880,08 Barcelona,NaN,Males,All ages,2024,2.870.721
1064881,08 Barcelona,NaN,Males,All ages,2023,2.830.260
1064882,08 Barcelona,NaN,Males,All ages,2022,2.783.698
1064883,08 Barcelona,NaN,Males,All ages,2021,2.785.890
1064884,08 Barcelona,NaN,Males,0 years old,2024,20.185
...,...,...,...,...,...,...
1066099,08 Barcelona,NaN,Total,99 years old,2021,1.096
1066100,08 Barcelona,NaN,Total,100 years or more,2024,1.982
1066101,08 Barcelona,NaN,Total,100 years or more,2023,1.807
1066102,08 Barcelona,NaN,Total,100 years or more,2022,1.707


### We have information about provinces without the municipalities, we should drop them

In [63]:
df_raw[df_raw['Provinces'] == "25 Lleida"].sample(5)

,Provinces,Municipalities,Sex,Age,Periodo,Total
4762952,25 Lleida,25051 Bellver de Cerdanya,Males,91 years old,2024,2
4930511,25 Lleida,25216 Talavera,Males,58 years old,2021,3
4716455,25 Lleida,25010 Alcanó,Males,94 years old,2021,1
4881875,25 Lleida,25167 Pinós,Females,37 years old,2021,2
4899116,25 Lleida,25182 Puigverd de Lleida,Females,64 years old,2024,3


In [64]:

prov_null = df_raw[["Provinces", "Municipalities"]]

prov_null[(prov_null["Provinces"].isnull()) & (prov_null["Municipalities"].isnull())]

,Provinces,Municipalities
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1219,NaN,NaN
1220,NaN,NaN
1221,NaN,NaN
1222,NaN,NaN


In [65]:

prov_null[(prov_null["Provinces"].isnull())]

,Provinces,Municipalities
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1219,NaN,NaN
1220,NaN,NaN
1221,NaN,NaN
1222,NaN,NaN


In [66]:
df_raw[df_raw['Total'].isnull()].sample(30)

,Provinces,Municipalities,Sex,Age,Periodo,Total
9348297,48 Bizkaia,48916 NA,Females,49 years old,2023,NaN
9348081,48 Bizkaia,48916 NA,Males,97 years old,2023,NaN
9347761,48 Bizkaia,48916 NA,Males,17 years old,2023,NaN
9348169,48 Bizkaia,48916 NA,Females,17 years old,2023,NaN
9348605,48 Bizkaia,48916 NA,Total,24 years old,2023,NaN
9348001,48 Bizkaia,48916 NA,Males,77 years old,2023,NaN
9348726,48 Bizkaia,48916 NA,Total,54 years old,2022,NaN
9348345,48 Bizkaia,48916 NA,Females,61 years old,2023,NaN
9347795,48 Bizkaia,48916 NA,Males,25 years old,2021,NaN
9348695,48 Bizkaia,48916 NA,Total,46 years old,2021,NaN


In [67]:
df_raw.sample(20)

,Provinces,Municipalities,Sex,Age,Periodo,Total
2572154,14 Córdoba,14038 Lucena,Females,29 years old,2022,262
8143233,44 Teruel,44067 Castelnou,Total,91 years old,2023,0
6267840,34 Palencia,34036 Brañosera,Total,35 years old,2024,1
7304716,40 Segovia,40073 Escalona del Prado,Total,72 years old,2024,5
9500064,49 Zamora,49134 Muelas de los Caballeros,Females,47 years old,2024,1
4562564,24 León,24103 Oencia,Females,76 years old,2024,2
6987763,37 Salamanca,"37346 Veguillas, Las",Total,87 years old,2021,4
2997925,16 Cuenca,16263 Villar de Olalla,Males,86 years old,2023,3
4198973,22 Huesca,22126 Ibieca,Females,60 years old,2023,0
9011632,47 Valladolid,47064 Fontihoyuelo,Females,33 years old,2024,1


In [68]:
df_raw = df_raw.dropna(subset=["Provinces", "Municipalities", "Total"], how="any")

In [69]:
df_2024 = df_raw[df_raw['Periodo'] == 2024].copy()

In [70]:
df_2024.reset_index(drop=True, inplace=True)

In [71]:
idx = pd.Index(["Provinces", "Municipalities", "Sex", "Age", "Periodo", "Total"])
eng = pd.Index(["province", "municipality", "sex", "age", "year", "total"])

df_2024.rename(columns=dict(zip(idx, eng)), inplace=True)

### Get the correct types

In [72]:
df_2024["total"] = pd.to_numeric(df_2024["total"].str.replace(".", "", regex=False)).astype("int32")

In [73]:
df_2024 = df_2024[df_2024["age"] != "All ages"]

In [74]:
(df_2024["age"].str.contains(r"^\d"))

1          True
2          True
3          True
4          True
5          True
           ... 
2488387    True
2488388    True
2488389    True
2488390    True
2488391    True
Name: age, Length: 2463996, dtype: bool

In [75]:
df_2024.head()

,province,municipality,sex,age,year,total
1,01 Araba/Álava,01001 Alegría-Dulantzi,Males,0 years old,2024,11
2,01 Araba/Álava,01001 Alegría-Dulantzi,Males,1 year old,2024,9
3,01 Araba/Álava,01001 Alegría-Dulantzi,Males,2 years old,2024,15
4,01 Araba/Álava,01001 Alegría-Dulantzi,Males,3 years old,2024,12
5,01 Araba/Álava,01001 Alegría-Dulantzi,Males,4 years old,2024,9


In [76]:
from scripts.utils import split_column_at
df_2024["age"] = split_column_at(df_2024, "age", " ", index=0)
df_2024["province"] = split_column_at(df_2024, "province", " ", index=1)
df_2024["cprov"] = split_column_at(df_2024, "province", " ", index=0)
df_2024["cmun"] = split_column_at(df_2024, "municipality", " ", index=0)
df_2024["municipality"] = split_column_at(df_2024, "municipality", " ", index=1)

In [77]:
df_2024

,province,municipality,sex,age,year,total,cprov,cmun
1,Araba/Álava,Alegría-Dulantzi,Males,0,2024,11,Araba/Álava,01001
2,Araba/Álava,Alegría-Dulantzi,Males,1,2024,9,Araba/Álava,01001
3,Araba/Álava,Alegría-Dulantzi,Males,2,2024,15,Araba/Álava,01001
4,Araba/Álava,Alegría-Dulantzi,Males,3,2024,12,Araba/Álava,01001
5,Araba/Álava,Alegría-Dulantzi,Males,4,2024,9,Araba/Álava,01001
...,...,...,...,...,...,...,...,...
2488387,Melilla,Melilla,Total,96,2024,19,Melilla,52001
2488388,Melilla,Melilla,Total,97,2024,13,Melilla,52001
2488389,Melilla,Melilla,Total,98,2024,11,Melilla,52001
2488390,Melilla,Melilla,Total,99,2024,3,Melilla,52001


In [78]:
df_2024.drop("cprov", axis = 1, inplace=True)

In [79]:
df_2024["age"] = pd.to_numeric(df_2024["age"].str.replace(".", "", regex=False)).astype("int32")

In [80]:
cleaner = AccentCleaner([df_2024], ['municipality', 'province'])
cleaner.cleanAccents()

In [81]:
df_2024.sample(10)

,province,municipality,sex,age,year,total,cmun,municipality_clean,province_clean
1503720,Ourense,Coles,Males,35,2024,15,32026,coles,ourense
1527355,Asturias,Cangas,Females,6,2024,30,33011,cangas,asturias
2076817,Toledo,Alcaudete,Total,96,2024,4,45006,alcaudete,toledo
2014323,Teruel,Berge,Total,26,2024,2,44040,berge,teruel
1567104,Palencia,Guardo,Males,77,2024,27,34080,guardo,palencia
2031521,Teruel,Frías,Total,88,2024,1,44109,frias,teruel
1955276,Tarragona,Blancafort,Total,37,2024,5,43029,blancafort,tarragona
918347,Guadalajara,"Mierla,",Males,40,2024,0,19182,"mierla,",guadalajara
38487,Albacete,Valdeganga,Total,32,2024,26,02075,valdeganga,albacete
1015906,Huesca,Alcalá,Total,87,2024,5,22014,alcala,huesca


In [82]:
df_2024["province"] = split_column_at(df_2024, "province", " ", index=0)

In [83]:
df_2024["cmun"] = pd.to_numeric(df_2024["cmun"].str.replace(".", "", regex=False)).astype("int32")

In [84]:
df_2024.sample(10)

,province,municipality,sex,age,year,total,cmun,municipality_clean,province_clean
105371,Almería,Oria,Females,4,2024,9,4070,oria,almeria
1736297,Salamanca,Vilvestre,Males,52,2024,3,37350,vilvestre,salamanca
1211138,Lleida,Conca,Total,91,2024,3,25161,conca,lleida
298456,Barcelona,Martorell,Females,3,2024,107,8114,martorell,barcelona
606129,Ciudad,Ciudad,Total,44,2024,1142,13034,ciudad,ciudad
405158,Burgos,Madrigal,Males,13,2024,1,9196,madrigal,burgos
771390,Girona,Foixà,Total,65,2024,5,17068,foixà,girona
1911273,Soria,Deza,Total,98,2024,0,42076,deza,soria
2412268,Zaragoza,Berdejo,Males,69,2024,1,50047,berdejo,zaragoza
1589628,Palencia,Sotobañado,Total,59,2024,4,34176,sotobanado,palencia


In [85]:
df_2024.to_csv("../data/large_files/filtered_age.csv", index=False)

## Grouping


In [86]:
df_ages_bined = pd.read_csv("../data/large_files/filtered_age.csv")

df_ages_bined.shape
df_ages_bined.head()

,province,municipality,sex,age,year,total,cmun,municipality_clean,province_clean
0,Araba/Álava,Alegría-Dulantzi,Males,0,2024,11,1001,alegria-dulantzi,araba/alava
1,Araba/Álava,Alegría-Dulantzi,Males,1,2024,9,1001,alegria-dulantzi,araba/alava
2,Araba/Álava,Alegría-Dulantzi,Males,2,2024,15,1001,alegria-dulantzi,araba/alava
3,Araba/Álava,Alegría-Dulantzi,Males,3,2024,12,1001,alegria-dulantzi,araba/alava
4,Araba/Álava,Alegría-Dulantzi,Males,4,2024,9,1001,alegria-dulantzi,araba/alava


In [87]:
df_demographics = df_ages_bined.query("sex != 'Total'")

In [88]:
df_demographics.isna().sum()

province                0
municipality          202
sex                     0
age                     0
year                    0
total                   0
cmun                    0
municipality_clean      0
province_clean          0
dtype: int64

In [89]:
def assign_age_group(age):
    if age <= 17:
        return '0-17'
    elif age <= 24:
        return '18-24'
    elif age <= 34:
        return '25-34'
    elif age <= 54:
        return '35-54'
    else:
        return '55+'

df_demographics['age_group'] = df_demographics['age'].apply(assign_age_group)

/var/folders/w9/bt5d4t050m7fd91t2r7l1mlh0000gn/T/ipykernel_62581/3734956762.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_demographics['age_group'] = df_demographics['age'].apply(assign_age_group)


In [90]:
df_demographics["age_group"].unique()

array(['0-17', '18-24', '25-34', '35-54', '55+'], dtype=object)

In [91]:
# 2. Pivot the table to have one column per age group
#    We group by 'cmun' (or whichever municipality identifier you prefer)
pivot_df = df_demographics.pivot_table(
    index='cmun',
    columns='age_group',
    values='total',
    aggfunc='sum',
    fill_value=0  # fill missing combinations with 0
).reset_index()


In [92]:
age_group_order = ['0-17', '18-24', '25-34', '35-54', '55+']
pivot_df = pivot_df[['cmun'] + [grp for grp in age_group_order if grp in pivot_df.columns]]
pivot_df["total_population"] = pivot_df["0-17"] + pivot_df["18-24"] + pivot_df["25-34"] + pivot_df["35-54"] + pivot_df["55+"]

pivot_df

age_group,cmun,0-17,18-24,25-34,35-54,55+,total_population
0,1001,615,250,268,1005,827,2965
1,1002,1826,588,862,2961,4075,10312
2,1003,218,113,108,349,592,1380
3,1004,352,127,146,576,655,1856
4,1006,42,20,18,96,70,246
...,...,...,...,...,...,...,...
8127,50901,6,8,16,37,103,170
8128,50902,4,2,4,33,42,85
8129,50903,405,224,320,789,1114,2852
8130,51001,18151,8386,10785,23963,21894,83179


In [93]:
pivot_df_sex = df_demographics.pivot_table(
    index='cmun',
    columns='sex',
    values='total',
    aggfunc='sum',
    fill_value=0  # fill missing combinations with 0
).reset_index()

In [94]:
df_demographics["sex"].unique()

array(['Males', 'Females'], dtype=object)

In [95]:
sex_group_order = ['Males', 'Females']

pivot_df_sex = pivot_df_sex[['cmun'] + [grp for grp in sex_group_order if grp in pivot_df_sex.columns]]
pivot_df_sex.rename(columns={"Males": "male", "Females": "female"}, inplace=True)

pivot_df_sex["total_sex"] = pivot_df_sex["male"] + pivot_df_sex["female"]
pivot_df_sex

sex,cmun,male,female,total_sex
0,1001,1525,1440,2965
1,1002,5134,5178,10312
2,1003,709,671,1380
3,1004,914,942,1856
4,1006,127,119,246
...,...,...,...,...
8127,50901,104,66,170
8128,50902,43,42,85
8129,50903,1444,1408,2852
8130,51001,41957,41222,83179


In [96]:
df_demographics_combined = pivot_df.merge(df_demographics, on='cmun', how='left')
df_demographics_combined = pivot_df_sex.merge(df_demographics_combined, on='cmun', how='left')

df_demographics_combined


,cmun,male,female,total_sex,0-17,18-24,25-34,35-54,55+,total_population,province,municipality,sex,age,year,total,municipality_clean,province_clean,age_group
0,1001,1525,1440,2965,615,250,268,1005,827,2965,Araba/Álava,Alegría-Dulantzi,Males,0,2024,11,alegria-dulantzi,araba/alava,0-17
1,1001,1525,1440,2965,615,250,268,1005,827,2965,Araba/Álava,Alegría-Dulantzi,Males,1,2024,9,alegria-dulantzi,araba/alava,0-17
2,1001,1525,1440,2965,615,250,268,1005,827,2965,Araba/Álava,Alegría-Dulantzi,Males,2,2024,15,alegria-dulantzi,araba/alava,0-17
3,1001,1525,1440,2965,615,250,268,1005,827,2965,Araba/Álava,Alegría-Dulantzi,Males,3,2024,12,alegria-dulantzi,araba/alava,0-17
4,1001,1525,1440,2965,615,250,268,1005,827,2965,Araba/Álava,Alegría-Dulantzi,Males,4,2024,9,alegria-dulantzi,araba/alava,0-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642659,52001,43252,42733,85985,21086,8706,11696,23407,21090,85985,Melilla,Melilla,Females,96,2024,10,melilla,melilla,55+
1642660,52001,43252,42733,85985,21086,8706,11696,23407,21090,85985,Melilla,Melilla,Females,97,2024,11,melilla,melilla,55+
1642661,52001,43252,42733,85985,21086,8706,11696,23407,21090,85985,Melilla,Melilla,Females,98,2024,9,melilla,melilla,55+
1642662,52001,43252,42733,85985,21086,8706,11696,23407,21090,85985,Melilla,Melilla,Females,99,2024,1,melilla,melilla,55+


In [97]:
df_demographics_combined = df_demographics_combined.drop(["year", "municipality", "province", "total_sex"], axis = 1)



In [98]:
df_demographics_combined

,cmun,male,female,0-17,18-24,25-34,35-54,55+,total_population,sex,age,total,municipality_clean,province_clean,age_group
0,1001,1525,1440,615,250,268,1005,827,2965,Males,0,11,alegria-dulantzi,araba/alava,0-17
1,1001,1525,1440,615,250,268,1005,827,2965,Males,1,9,alegria-dulantzi,araba/alava,0-17
2,1001,1525,1440,615,250,268,1005,827,2965,Males,2,15,alegria-dulantzi,araba/alava,0-17
3,1001,1525,1440,615,250,268,1005,827,2965,Males,3,12,alegria-dulantzi,araba/alava,0-17
4,1001,1525,1440,615,250,268,1005,827,2965,Males,4,9,alegria-dulantzi,araba/alava,0-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642659,52001,43252,42733,21086,8706,11696,23407,21090,85985,Females,96,10,melilla,melilla,55+
1642660,52001,43252,42733,21086,8706,11696,23407,21090,85985,Females,97,11,melilla,melilla,55+
1642661,52001,43252,42733,21086,8706,11696,23407,21090,85985,Females,98,9,melilla,melilla,55+
1642662,52001,43252,42733,21086,8706,11696,23407,21090,85985,Females,99,1,melilla,melilla,55+


In [99]:
df_demographics_combined = df_demographics_combined[["cmun", "municipality_clean", "province_clean", "0-17", "18-24", "25-34", "35-54","55+", "male", "female","total_population"]]

In [100]:
df_demographics_combined

,cmun,municipality_clean,province_clean,0-17,18-24,25-34,35-54,55+,male,female,total_population
0,1001,alegria-dulantzi,araba/alava,615,250,268,1005,827,1525,1440,2965
1,1001,alegria-dulantzi,araba/alava,615,250,268,1005,827,1525,1440,2965
2,1001,alegria-dulantzi,araba/alava,615,250,268,1005,827,1525,1440,2965
3,1001,alegria-dulantzi,araba/alava,615,250,268,1005,827,1525,1440,2965
4,1001,alegria-dulantzi,araba/alava,615,250,268,1005,827,1525,1440,2965
...,...,...,...,...,...,...,...,...,...,...,...
1642659,52001,melilla,melilla,21086,8706,11696,23407,21090,43252,42733,85985
1642660,52001,melilla,melilla,21086,8706,11696,23407,21090,43252,42733,85985
1642661,52001,melilla,melilla,21086,8706,11696,23407,21090,43252,42733,85985
1642662,52001,melilla,melilla,21086,8706,11696,23407,21090,43252,42733,85985


In [101]:
df_demographics_combined = df_demographics_combined.drop_duplicates()
df_demographics_combined

,cmun,municipality_clean,province_clean,0-17,18-24,25-34,35-54,55+,male,female,total_population
0,1001,alegria-dulantzi,araba/alava,615,250,268,1005,827,1525,1440,2965
202,1002,amurrio,araba/alava,1826,588,862,2961,4075,5134,5178,10312
404,1003,aramaio,araba/alava,218,113,108,349,592,709,671,1380
606,1004,artziniega,araba/alava,352,127,146,576,655,914,942,1856
808,1006,arminon,araba/alava,42,20,18,96,70,127,119,246
...,...,...,...,...,...,...,...,...,...,...,...
1641654,50901,biel,zaragoza,6,8,16,37,103,104,66,170
1641856,50902,marracos,zaragoza,4,2,4,33,42,43,42,85
1642058,50903,villamayor,zaragoza,405,224,320,789,1114,1444,1408,2852
1642260,51001,ceuta,ceuta,18151,8386,10785,23963,21894,41957,41222,83179


In [102]:
df_demographics_combined.to_csv("../data/processed/filtered_demographics.csv", index=False)